# Using spaCy for Capstone Project
### This includes more cleaning and EDA with some modeling practice 

https://www.geeksforgeeks.org/python-sort-python-dictionaries-by-key-or-value/   
https://medium.com/@armandj.olivares/building-nlp-content-based-recommender-systems-b104a709c042      
https://towardsdatascience.com/so-whats-spacy-ad65aa1949e0   
The following links from spacy.io were used to help create this system (note there are many links based off these that are used, but these are the main ones)  
https://spacy.io/usage   
https://spacy.io/usage/spacy-101   
https://spacy.io/usage/vectors-similarity   
https://spacy.io/api/span#vector    
https://spacy.io/api/vectors     
  

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import spacy

from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [3]:
nlp_sm = spacy.load('en_core_web_sm')
nlp_lg = spacy.load('en_core_web_lg')

In [4]:
data = pd.read_csv('./data/chewy.csv')

In [5]:
data.head()

,title,price,descriptions,key_benefits,rating,reviews,subcat,cat,combined_text,combined_sent_len,combined_word_leng,tokenized_combined,token_vectors,avg_vector
0,Nylabone Teething Pacifier Puppy Chew Toy,3.59,'Every puppy needs a pacifier to soothe teethi...,Designed to encourage positive play and teach ...,4.2,I do not know how to rate this. The puppy this...,moderate,chew toys,Every puppy needs a pacifier to soothe teethin...,50,1146,"['every', 'puppy', 'needs', 'pacifier', 'sooth...","[-0.04003238, 0.16376925, -0.17635255, -0.0941...",0.002920
1,"KONG Puppy Dog Toy, Color Varies",6.99,"""The Puppy KONG dog toy is customized for a gr...",Unpredictable bounce is great for energetic pu...,4.3,"I have had dozens of dogs over the years, and ...",moderate,chew toys,"""The Puppy KONG dog toy is customized for a gr...",43,964,"['puppy', 'kong', 'dog', 'toy', 'customized', ...","[-0.04645431, 0.17621087, -0.15408944, -0.0865...",0.002565
2,Petstages Dogwood Tough Dog Chew Toy,8.83,"""Chewing is a natural behavior in all dogs, as...",Chew toy that combines real wood with syntheti...,4.2,"Our 8 month old, shepherd/mastiff has had this...",moderate,chew toys,"""Chewing is a natural behavior in all dogs, as...",48,1144,"['chewing', 'natural', 'behavior', 'dogs', 'he...","[-0.043032583, 0.16056213, -0.17279097, -0.084...",0.005094
3,Nylabone Teething Rings Puppy Chew Toy,6.57,'Great for teething and tugging! The Puppy Tee...,Designed to encourage positive play and teach ...,4.1,I have a small shih tzu who absolutely loves t...,moderate,chew toys,Great for teething and tugging! The Puppy Teet...,32,772,"['great', 'teething', 'tugging', 'puppy', 'tee...","[-0.040631622, 0.15921111, -0.15474631, -0.073...",0.004021
4,Nylabone Puppy Teething X Bone Beef Flavored P...,6.89,'Curious puppies have met their match with the...,Non-edible dog toy is made for teething puppie...,4.1,This is a great shoe for a tiny puppy in there...,moderate,chew toys,Curious puppies have met their match with the ...,25,804,"['curious', 'puppies', 'met', 'match', 'nylabo...","[-0.03072327, 0.15317601, -0.14757921, -0.0565...",0.003230


# This next section will be usefull code for the actual recommendation system 

### User Input-- What will the user need to do?
The user input will be a short paragraph answering the following the questions to the best of their abilities:
 - What type of dog do you have?
 - Is your dog a puppy, or older?
 - Is your dog super energetic and playful or more calm?
 - Would you say your dog is smarter than most dogs?
 - Does your dog like to chase after toys, cuddle toys, or chew on them? 
 - Does your dog get pretty bored after playing with a toy for a bit? 
 - Does your dog already have an ideal toy that is their absolute favorite? (If yes, describe it.)   
 Try to describe what your dog IS rather what you dog IS NOT so that the model works better. 
 
The paragraph of text describing their dog will be tokenized and stopwords removed so that it can then be compared to tokenized text for every dog toy to find the most similar toys based off the text. 

The text that I will use to create and test this will be about one of my own dogs Colby:  
An okay example:   
"Colby is 9-year-old retriever. For the most part, he is pretty mellow, but sometimes he gets bursts of energy, especially when we go swimming and play fetch in the pool with a ball. He also really like treats and is very food oriented. I wouldn't say he's very smart and definielty prefers to chase toys more than cuddle or chew on them. He will get bored with a toy after a bit. His favorite toy is probably a ball and not sqeaky."

A better example:
"Colby is 9-year-old retriever. For the most part, he is pretty mellow, but sometimes he gets bursts of energy, especially when we go swimming and play fetch in the pool with a ball. He also really like treats and is very food oriented. He is pretty dumn and definielty prefers to chase toys. He will get bored with a toy after a bit. His favorite toy is probably a ball and not sqeaky."

A third example: 
"Eli is a miniature dachshund that loves to run around and snuggle with soft toys. She is very active but also loves to cuddle and loves toys." 

In [23]:
# tokenizer = RegexpTokenizer(r'\w+')
# stopwords = stopwords.words('english') + ['dog', 'toy', 'dogs', 'toys', 'one']

In [94]:
# User input cleaning: 
user_input_1 = "Colby is 9-year-old golden retriever. For the most part, he is pretty mellow, but sometimes he gets bursts of energy, especially when we go swimming and play fetch in the pool with a ball. He also really like treats and is very foot oriented. I wouldn't say he's very smart and definielty prefers to chase toys more than cuddle or chew on them. He will get bored with a toy after a bit. His favorite toy is probably a ball and not sqeaky."
user_input_2 = "Colby is 9-year-old retriever. For the most part, he is pretty mellow, but sometimes he gets bursts of energy, especially when we go swimming and play fetch in the pool with a ball. He also really like treats and is very food oriented. He is pretty dumn and definielty prefers to chase toys. He will get bored with a toy after a bit. His favorite toy is probably a ball and not sqeaky."
user_input_3 = "Eli is a miniature dachshund that loves to run around and snuggle with soft toys. She is very active but also loves to cuddle and loves toys." 
user_input_4 = "My dog is incredibly smart but lazy and loves plush toys to cuddle with."
# user_doc = nlp_lg(user_input)

# user_tokens = tokenizer.tokenize(str(user_doc).lower())
# user_text = [token for token in user_tokens if token not in stopwords]

# user_no_stop_words
# user_text = []
# for token in user_doc:
#     tokenizer.tokenize(str(token).lower())
# user_text

In [ ]:
# combined_tokens = tokenizer.tokenize(df['combined_text'][i].lower())
# no_stop_words = [token for token in combined_tokens if token not in stopwords.words('english')]

In [28]:
# https://medium.com/@armandj.olivares/building-nlp-content-based-recommender-systems-b104a709c042

list_docs = []
for i in range(len(data)):
    if data['combined_text'][i] != '':
        doc = nlp_lg(str(data['combined_text'][i]) + "'")
        list_docs.append(doc)

In [98]:
len(list_docs)

2399

In [101]:
# https://medium.com/@armandj.olivares/building-nlp-content-based-recommender-systems-b104a709c042
# This will take in what the users says and then score it based off that 
# I have added to this as well

def calculate_similarity_with_spacy(nlp, df, user_text, n=6):
    # Calculate similarity with Spacy 
    list_sim = []
    toy_score = []
    doc1 = nlp(user_text + "'")
    vectors = doc1.vector # not quite sure if this will be useful bc not sure how to use the .most_similar here
    for i in df.index:
        try: 
            doc2 = list_docs[i]
            score = doc1.similarity(doc2)
            list_sim.append((doc1, doc2, score))
            toy_score.append((score, df['title'][i], df['cat'][i], df['description']))
        except:
            continue
    return toy_score #, vect_list  #list_sim

Running it on the okay example

In [56]:
ranking_list = calculate_similarity_with_spacy(nlp_lg, data, user_text=user_input_1)
# ranking_list

In [103]:
sorted(ranked_dict, reverse=True)

[0.9725215820014242,
 0.9716290016880776,
 0.9704434155124235,
 0.9702837070088358,
 0.9691803954768489,
 0.968903733435692,
 0.9688948048534278,
 0.9688758294095094,
 0.9687495983530308,
 0.9671432535161417]

In [146]:
def dog_toy_recommender(nlp, data, user_text):
    ranking_list = calculate_similarity_with_spacy(nlp_lg, data, user_text=user_text)
    ranked_dict = {}
    for i, score_toy in enumerate(ranking_list):
    #     print(i, score_toy[0], ranking_list[i][2])
        ranked_dict[score_toy[0]] = {i : [ranking_list[i][2], score_toy[1]]}
    final_dict = {}
    for i in sorted(ranked_dict, reverse=True) : 
        final_dict[i] =  ranked_dict[i]
    return final_dict

In [147]:
def run_recommender(nlp, data, user_text):
    toys = dog_toy_recommender(nlp_lg, data, user_text=user_text)
    for i in list(toys)[:10]:
        print(i, toys[i])

In [148]:
run_recommender(nlp_lg, data, user_text=user_input_1)

0.9853530786203206 {1185: ['plush toys', 'GoDog Grumpy Fat Dino Chew Guard Squeaky Plush Dog Toy']}
0.9852264630933704 {1426: ['plush toys', 'Outward Hound Floppyz Donkey Squeaky Plush Dog Toy']}
0.9851527160042408 {785: ['interactive toys', 'Jolly Pets Teaser Ball Dog Toy, Blue']}
0.9850387127577117 {646: ['interactive toys', 'PetSafe Sportsmen Football Treat Dispensing Tough Dog Chew Toy']}
0.9849370610053354 {890: ['plush toys', 'Frisco Muscle Plush Squeaking Tiger Dog Toy']}
0.9848946180203382 {902: ['plush toys', 'Ethical Pet Soccer Ball Squeaky Plush Dog Toy']}
0.984844510710498 {2033: ['plush toys', 'Mighty Squeaky Stuffing-Free Plush Ball Dog Toy, Orange, Large']}
0.9847632216384811 {318: ['fetch toys', 'JW Pet Whirlwheel Flying Disk Dog Toy, Color Varies, Large']}
0.9845452458744287 {1698: ['plush toys', 'Outward Hound Floppyz Chicken Squeaky Plush Dog Toy']}
0.9844364596118705 {1106: ['plush toys', "Hyper Pet Disney's Nightmare Before Christmas Jack Plush Dog Toy"]}


In [149]:
# ranking_list = calculate_similarity_with_spacy(nlp_lg, data, user_text=user_input_1)
# # , vectors

# # queries = numpy.asarray([numpy.random.uniform(-1, 1, (300,))])
# # most_similar = nlp_lg.vocab.vectors.most_similar(np.array(vectors), n=10)

# ranked_dict = {}
# for i, score_toy in enumerate(ranking_list[:10]):
# #     print(i, score_toy[0], ranking_list[i][2])
#     ranked_dict[score_toy[0]] = {i : [score_toy[1], ranking_list[i][2]]}

# # https://www.geeksforgeeks.org/python-sort-python-dictionaries-by-key-or-value/
# final_dict = {}
# for i in sorted(ranked_dict, reverse=True) : 
#     print ((i, ranked_dict[i]), end ='\n')
# #     final_dict[i] = ranked_dict[i]
# # final_dict

In [150]:
run_recommender(nlp_lg, data, user_text=user_input_2)

0.9800368148543878 {646: ['interactive toys', 'PetSafe Sportsmen Football Treat Dispensing Tough Dog Chew Toy']}
0.9794736895011875 {2033: ['plush toys', 'Mighty Squeaky Stuffing-Free Plush Ball Dog Toy, Orange, Large']}
0.9790931578882266 {1185: ['plush toys', 'GoDog Grumpy Fat Dino Chew Guard Squeaky Plush Dog Toy']}
0.9790702211002554 {619: ['interactive toys', 'Planet Dog Orbee-Tuff Mazee Puzzle Dog Toy']}
0.9788167069914844 {318: ['fetch toys', 'JW Pet Whirlwheel Flying Disk Dog Toy, Color Varies, Large']}
0.9787709201069285 {902: ['plush toys', 'Ethical Pet Soccer Ball Squeaky Plush Dog Toy']}
0.9787447615411639 {1426: ['plush toys', 'Outward Hound Floppyz Donkey Squeaky Plush Dog Toy']}
0.9786543916804369 {24: ['chew toys', 'Outward Hound Triple Jack Squeaky Dog Chew Toy']}
0.978596547503262 {1022: ['plush toys', "Multipet Look Who's Talking Sheep Plush Dog Toy"]}
0.9785280374824761 {2018: ['plush toys', 'Chuckit! Indoor Flying Squirrel Dog Toy']}


In [151]:
run_recommender(nlp_lg, data, user_text=user_input_3)

0.9551266090697785 {1967: ['plush toys', 'Charming Pet Lumber Jackerz Dog Toy, Bunny']}
0.9527661083164954 {1381: ['plush toys', 'Pet Qwerks Barking Puppy Sound Plush Dog Toy']}
0.9527034717669484 {1410: ['plush toys', 'ZippyPaws Finn the Fish Grunterz Dog Toy']}
0.9520430500174887 {1566: ['plush toys', "Li'l Pals Plush Dog Toy"]}
0.9512635740630994 {1710: ['plush toys', 'Fab Dog Floppy Panda Squeaky Plush Dog Toy, Small']}
0.9511907914658626 {1014: ['plush toys', 'Ethical Pet Lil Spots Squeaky Plush Blanket Puppy Toy, Color/Character Varies']}
0.9510905293234249 {1261: ['plush toys', "Charming Pet Squeakin' Squiggles Elephant Squeaky Plush Dog Toy"]}
0.9509657315015633 {1369: ['plush toys', 'ZippyPaws Yellow Lab Squeakie Buns Dog Toy']}
0.9507440410586208 {1438: ['plush toys', 'Ethical Pet Fun Food Ruffus Chips Squeaky Plush Dog Toy']}
0.9505427627459778 {1592: ['plush toys', 'Charming Pet Cuddle Tugs Pig Squeaky Plush Dog Toy']}


In [152]:
run_recommender(nlp_lg, data, user_text=user_input_4)

0.9437791185063656 {1739: ['plush toys', 'Ethical Pet Cuddle Bunnies Squeaky Plush Dog Toy, Character Varies']}
0.9429259507659253 {1470: ['plush toys', 'Remington Plush Dog Toy, Shotgun Shell']}
0.9414243167216859 {1669: ['plush toys', 'Ethical Pet Corduroy Hedgehog Squeaky Plush Dog Toy, Character Varies']}
0.9413042386244863 {1822: ['plush toys', 'Zoobilee Skins Plush Puppy Toy, 2 count']}
0.9409792765067402 {1967: ['plush toys', 'Charming Pet Lumber Jackerz Dog Toy, Bunny']}
0.940000162202191 {1261: ['plush toys', "Charming Pet Squeakin' Squiggles Elephant Squeaky Plush Dog Toy"]}
0.9370589843751561 {1851: ['plush toys', 'All For Paws Monster Bunch Stick Dog Toy']}
0.9365866182809068 {1576: ['plush toys', 'Pet Qwerks Pig Sound Plush Dog Toy']}
0.9364032760174563 {1849: ['plush toys', 'Pet Shop by Fringe Studio Donut Leave Me Hangin Squeaky Plush Dog Toy']}
0.9357075513104911 {1550: ['plush toys', 'Ethical Pet Fun Food Chicken Leg Squeaky Plush Dog Toy']}


In [153]:
run_recommender(nlp_lg, data, user_text='pattycake is a cockapoo. she is older and very calm. she likes to stay indoors, and bark at squirrels from the window')

0.9463557045453683 {889: ['plush toys', 'Frisco Fur Really Real Squirrel Dog Toy']}
0.943792811148411 {1114: ['plush toys', 'Smart Pet Love New Puppy Starter Kit']}
0.9435811650198336 {1179: ['plush toys', 'Multipet Swingin Slevins Squeaky Plush Dog Toy']}
0.9423917696231578 {1381: ['plush toys', 'Pet Qwerks Barking Puppy Sound Plush Dog Toy']}
0.9423276207395921 {1127: ['plush toys', 'Petstages Cuddle Pal Sloth Dog Plush Toy']}
0.9415614400297996 {863: ['plush toys', 'Smart Pet Love Snuggle Puppy Behavioral Aid Dog Toy']}
0.9412923771116017 {888: ['plush toys', "Frisco Puppy Lil' Romps Bungee Elephant Dog Toy"]}
0.9408595604481523 {1162: ['plush toys', 'Ethical Pet Woodland Collection Squirrel Squeaky Plush Dog Toy']}
0.9406765269641763 {941: ['plush toys', 'Frisco Textured Plush Squeaking Bunny Dog Toy']}
0.9406349101788064 {1273: ['plush toys', 'Jolly Pets Tug-a-Mals Crab Dog Toy']}
